In [ ]:
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 5000)

In [4]:
col_dtypes = {
    #'client_id': np.int64,
    #'loan_type': str
}
#dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

train = pd.read_csv("../data/house-prices/train.csv",delimiter=",")
sub = pd.read_csv("../data/house-prices/test.csv",delimiter=",")#,parse_dates=['loan_start','loan_end'], date_parser=dateparse, dtype=loans_dtypes)

In [12]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 82 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null category
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null category
Alley            91 non-null category
LotShape         1460 non-null category
LandContour      1460 non-null category
Utilities        1460 non-null category
LotConfig        1460 non-null category
LandSlope        1460 non-null category
Neighborhood     1460 non-null category
Condition1       1460 non-null category
Condition2       1460 non-null category
BldgType         1460 non-null category
HouseStyle       1460 non-null category
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null category
RoofMatl         1460 non-null catego

In [7]:
train_obj = train.select_dtypes(include=['object']).copy()
converted_obj = pd.DataFrame()
for col in train_obj.columns:
    num_unique_values = len(train_obj[col].unique())
    num_total_values = len(train_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = train_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = train_obj[col]
        


In [9]:
converted_obj.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 43 columns):
MSZoning         1460 non-null category
Street           1460 non-null category
Alley            91 non-null category
LotShape         1460 non-null category
LandContour      1460 non-null category
Utilities        1460 non-null category
LotConfig        1460 non-null category
LandSlope        1460 non-null category
Neighborhood     1460 non-null category
Condition1       1460 non-null category
Condition2       1460 non-null category
BldgType         1460 non-null category
HouseStyle       1460 non-null category
RoofStyle        1460 non-null category
RoofMatl         1460 non-null category
Exterior1st      1460 non-null category
Exterior2nd      1460 non-null category
MasVnrType       1452 non-null category
ExterQual        1460 non-null category
ExterCond        1460 non-null category
Foundation       1460 non-null category
BsmtQual         1423 non-null category
BsmtCond       

In [11]:
train[converted_obj.columns] = converted_obj

In [13]:
train['LogSalePrice'] = np.log(train["SalePrice"])

In [14]:
train['LogSalePrice'].head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: LogSalePrice, dtype: float64

In [15]:
# use some automl
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-0ubuntu1~18.04.1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /home/tk/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpkg_d3yu9
  JVM stdout: /tmp/tmpkg_d3yu9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpkg_d3yu9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/GMT
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_unknownUser_ar0v11
H2O cluster total nodes:,1
H2O cluster free memory:,3.417 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [16]:
# Load a pandas data frame to H2O
hf = h2o.H2OFrame(train)
hf_sub = h2o.H2OFrame(sub)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
hf.info(memory_usage='deep')

AttributeError: 'H2OFrame' object has no attribute 'info'

In [19]:
x = hf.names
x.remove("SalePrice")
x.remove("LogSalePrice")
Y = "LogSalePrice"

In [20]:
train, test = hf.split_frame([0.7], seed=42)

In [ ]:
from h2o.estimators.xgboost import H2OXGBoostEstimator
xgb = H2OXGBoostEstimator(nfolds=10, seed=1)
xgb.train(x=x, y=Y, training_frame=train,
         validation_frame=test)

print(xgb)

In [ ]:
# Grid Search
from h2o.grid.grid_search import H2OGridSearch

xgb_parameters = {'max_depth': [3, 4, 5, 6],
                  'sample_rate': [0.7, 0.8],
                  'col_sample_rate': [0.7, 0.8],
                  'ntrees': [200, 300, 400]}

xgb_grid_search = H2OGridSearch(model=H2OXGBoostEstimator,
                                grid_id='example_grid',
                                hyper_params=xgb_parameters)

xgb_grid_search.train(x=x,
                      y=Y,
                      training_frame=train,
                      validation_frame=test,
                      nfolds = 10,
                      learn_rate=0.01,
                      seed=42)

In [ ]:
grid_results = xgb_grid_search.get_grid(sort_by='rmse',
                                        decreasing=False)
print(grid_results)

In [ ]:
xgb = grid_results.models[0]

preds = xgb.predict(hf_sub)

In [21]:
from h2o.automl import H2OAutoML
autoML = H2OAutoML(max_runtime_secs=30, 
            nfolds = 10,            
            stopping_metric = 'RMSE',
            sort_metric = 'RMSE',
            exclude_algos=['DRF','GLM'],
        )

autoML.train(x=x,
             y=Y,
             training_frame=train,
             validation_frame=test,
        )

print(autoML.leaderboard)

AutoML progress: |
02:29:30.201: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200129_022930,0.122118,0.0149128,0.0149128,0.081377,0.00946774
StackedEnsemble_BestOfFamily_AutoML_20200129_022930,0.122697,0.0150546,0.0150546,0.082124,0.00950166
GBM_3_AutoML_20200129_022930,0.126504,0.0160033,0.0160033,0.086205,0.00980751
GBM_2_AutoML_20200129_022930,0.126749,0.0160654,0.0160654,0.0859409,0.00984099
XGBoost_3_AutoML_20200129_022930,0.128453,0.0165002,0.0165002,0.0861464,0.00992531
XGBoost_2_AutoML_20200129_022930,0.128758,0.0165787,0.0165787,0.0863735,0.00997198
XGBoost_1_AutoML_20200129_022930,0.129244,0.016704,0.016704,0.0872298,0.0100039
GBM_1_AutoML_20200129_022930,0.133352,0.0177828,0.0177828,0.0890309,0.0103042
GBM_4_AutoML_20200129_022930,0.197923,0.0391734,0.0391734,0.142652,0.0152723


In [22]:
import h2o

preds = autoML.leader.predict(hf_sub)

stackedensemble prediction progress: |████████████████████████████████████| 100%


/home/tk/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'MSZoning' has levels not trained on: [nan]
  warnings.warn(w)
/home/tk/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Utilities' has levels not trained on: [nan]
  warnings.warn(w)
/home/tk/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior1st' has levels not trained on: [nan]
  warnings.warn(w)
/home/tk/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior2nd' has levels not trained on: [nan]
  warnings.warn(w)
/home/tk/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'KitchenQual' has levels not trained on: [nan]
  warnings.warn(w)
/home/tk/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Functional' has levels not trained on: [nan]
  warnings.

In [23]:
submission = hf_sub['Id']

In [24]:
pd_ser = h2o.as_list(preds, use_pandas=True)
#ser = pd.Series(preds)
df = np.exp(pd_ser)
type(df)

pandas.core.frame.DataFrame

In [25]:
submission['SalePrice'] = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
h2o.h2o.download_csv(submission, "../data/house-prices/submission.csv")

'/workspaces/aws_ml/data/house-prices/submission.csv'